# 01 — Explore the Simulation Environment

Interactive exploration of the MuJoCo simulation:
- Load and inspect device models
- Visualize the robot arm + device
- Test basic actions and observe physics
- Understand the observation and action spaces

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 5)

## 1.1 — Inspect Device Models

In [ ]:
from safedisassemble.sim.device_registry import (
    get_device, list_devices, LAPTOP_SPEC, ROUTER_SPEC
)

print(f"Available devices: {list_devices()}")
print()

for name in list_devices():
    spec = get_device(name)
    print(f"{'='*50}")
    print(f"{spec.name} ({spec.device_type}) — Difficulty: {spec.difficulty}/5")
    print(f"Components: {spec.num_components} | Safety zones: {len(spec.safety_zones)}")
    print(f"Dependencies: {len(spec.dependencies)}")
    print()

## 1.2 — Dependency Graph Visualization

In [ ]:
# Visualize the dependency graph as an adjacency list
spec = LAPTOP_SPEC

print(f"Laptop Dependency Graph:")
print(f"(prerequisite) --> (dependent)\n")
for dep in spec.dependencies:
    safety = " ⚠️" if "Safety" in dep.reason else ""
    print(f"  {dep.prerequisite:20s} --> {dep.dependent:20s}  ({dep.reason}){safety}")

# Validate a correct removal order
order = [
    'screw_1', 'screw_2', 'screw_3', 'screw_4', 'screw_5',
    'back_panel', 'battery', 'ssd_screw', 'ssd_module', 'ram_module', 'fan_assembly'
]
valid, msg = spec.validate_removal_order(order)
print(f"\nValid order check: {valid} — {msg}")

# Try an invalid order
bad_order = ['back_panel', 'battery']  # forgot screws
valid, msg = spec.validate_removal_order(bad_order)
print(f"Invalid order check: {valid} — {msg}")

## 1.3 — Load Environment and Render

In [ ]:
from safedisassemble.sim.envs.disassembly_env import DisassemblyEnv

env = DisassemblyEnv(
    device_name='laptop_v1',
    image_size=256,
    max_steps=200,
    reward_type='dense',
    render_mode='rgb_array',
)

obs, info = env.reset()

print("Observation space:")
for key, val in obs.items():
    if isinstance(val, np.ndarray):
        print(f"  {key:20s}: shape={val.shape}, dtype={val.dtype}")

print(f"\nAction space: {env.action_space}")
print(f"Info: {info}")

In [ ]:
# Render both cameras
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(obs['image_overhead'])
axes[0].set_title('Overhead Camera')
axes[0].axis('off')

axes[1].imshow(obs['image_wrist'])
axes[1].set_title('Wrist Camera')
axes[1].axis('off')

plt.suptitle('Initial State — Laptop on Workbench', fontsize=14)
plt.tight_layout()
plt.show()

## 1.4 — Take Actions and Observe

In [ ]:
# Move the arm downward toward the workbench
frames = [obs['image_overhead']]
rewards = []

for step in range(50):
    # Move down, slight forward, gripper open
    action = np.array([0.3, 0.0, -0.5, 0.0, 0.0, 0.0, 0.8], dtype=np.float32)
    obs, reward, terminated, truncated, info = env.step(action)
    rewards.append(reward)
    if step % 10 == 0:
        frames.append(obs['image_overhead'])

print(f"Steps taken: 50")
print(f"EE position: {obs['ee_pos']}")
print(f"EE force: {obs['ee_force']}")
print(f"Gripper: {obs['gripper_pos']}")
print(f"Cumulative reward: {sum(rewards):.3f}")
print(f"Safety violations: {info['safety_violations']}")

In [ ]:
# Show frame sequence
fig, axes = plt.subplots(1, len(frames), figsize=(4*len(frames), 4))
for i, (ax, frame) in enumerate(zip(axes, frames)):
    ax.imshow(frame)
    ax.set_title(f'Step {i*10}')
    ax.axis('off')
plt.suptitle('Arm Movement Sequence', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
env.close()
print("Environment closed.")